# Инференс моделей PyTorch 'pt' (yolov5)

In [ ]:
# conda activate yolov5

Вместо * необходимо подставить свои значения.

## Из репозитория

In [ ]:
# !git clone https://github.com/ultralytics/yolov5
%cd yolov5
!python detect.py \
	--weights best.pt \
    --source *.png \
    --conf-thres 0.001 \
    --iou-thres 0.45 \
	--img 640 \
    --project project_path

Результаты сохраняются по адресу: project_path\exp.

## Средствами PyTorch

In [ ]:
import os
import torch

In [ ]:
# загрузка модели
model = torch.hub.load('ultralytics/yolov5', 'custom', path='*.pt', force_reload=True)

In [3]:
# слои модели
model

AutoShape(
  (model): DetectMultiBackend(
    (model): DetectionModel(
      (model): Sequential(
        (0): Conv(
          (conv): Conv2d(3, 32, kernel_size=(6, 6), stride=(2, 2), padding=(2, 2))
          (act): SiLU(inplace=True)
        )
        (1): Conv(
          (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
          (act): SiLU(inplace=True)
        )
        (2): C3(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): Bottleneck(
              (cv1): Conv(
                (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1))
  

In [4]:
# так удобнее посмотреть слои
for name, para in model.named_parameters():
    print('{}: {}'.format(name, para.shape))

model.model.model.0.conv.weight: torch.Size([32, 3, 6, 6])
model.model.model.0.conv.bias: torch.Size([32])
model.model.model.1.conv.weight: torch.Size([64, 32, 3, 3])
model.model.model.1.conv.bias: torch.Size([64])
model.model.model.2.cv1.conv.weight: torch.Size([32, 64, 1, 1])
model.model.model.2.cv1.conv.bias: torch.Size([32])
model.model.model.2.cv2.conv.weight: torch.Size([32, 64, 1, 1])
model.model.model.2.cv2.conv.bias: torch.Size([32])
model.model.model.2.cv3.conv.weight: torch.Size([64, 64, 1, 1])
model.model.model.2.cv3.conv.bias: torch.Size([64])
model.model.model.2.m.0.cv1.conv.weight: torch.Size([32, 32, 1, 1])
model.model.model.2.m.0.cv1.conv.bias: torch.Size([32])
model.model.model.2.m.0.cv2.conv.weight: torch.Size([32, 32, 3, 3])
model.model.model.2.m.0.cv2.conv.bias: torch.Size([32])
model.model.model.3.conv.weight: torch.Size([128, 64, 3, 3])
model.model.model.3.conv.bias: torch.Size([128])
model.model.model.4.cv1.conv.weight: torch.Size([64, 128, 1, 1])
model.model.mo

In [ ]:
# инференс
img = '*.png'
results = model(img)
print(results)

In [ ]:
list(results.pred)

In [7]:
results.pandas().xyxy[0]

,xmin,ymin,xmax,ymax,confidence,class,name
0,412.377747,242.831482,435.799500,295.981079,0.633974,0,Corrosion
1,176.143661,396.686401,214.988174,508.654114,0.617267,0,Corrosion
2,403.028503,11.713810,427.832153,76.807175,0.303495,0,Corrosion
3,403.225586,578.997864,432.352905,640.000000,0.251476,0,Corrosion
4,404.885620,54.259666,427.078186,97.550575,0.251334,0,Corrosion


# Cохранение модели yolov5 в onnx

In [ ]:
# !git clone https://github.com/ultralytics/yolov5
%cd yolov5
!python export.py \
	--weights *.pt \
	--img 320 \
	--batch 1

# Инференс моделей onnx (yolov5)

## PyTorch

In [ ]:
import cv2
import numpy as np
import pandas as pd

# определите параметры детектирования
INPUT_WIDTH = 640 # ширина входных изображений
INPUT_HEIGHT = 640 # высота входных изображений
SCORE_THRESHOLD = 0.45 # порог уверенности модели классификации
NMS_THRESHOLD = 0.45 # порог площади перекрытия для исключения лишних рамок
CONFIDENCE_THRESHOLD = 0.5 # порог уверенности модели детектора

# укажите путь загрузки файла имен классов
classesFile = 'names.names'
# укажите путь к изображению
frame = cv2.imread('*.png')
# укажите путь к модели onnx
modelWeights = '*.onnx'

# Text parameters
FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.5
THICKNESS = 1
# Colors
BLACK  = (0,0,0)
BLUE   = (255,178,50)
YELLOW = (0,255,255)


def draw_label(im, label, x, y):
    """Draw text onto image at location."""
    cv2.putText(im, label, (x, y), FONT_FACE, FONT_SCALE, BLACK, THICKNESS, cv2.LINE_AA)


def pre_process(input_image, net):
      # Create a 4D blob from a frame.
      blob = cv2.dnn.blobFromImage(input_image, 1/255,  (INPUT_WIDTH, INPUT_HEIGHT), [0,0,0], 1, crop=False)
      # Sets the input to the network.
      net.setInput(blob)
      # Run the forward pass to get output of the output layers.
      outputs = net.forward(net.getUnconnectedOutLayersNames())

      return outputs


def post_process(input_image, outputs):
      # Lists to hold respective values while unwrapping.
      class_ids = []
      confidences = []
      boxes = []
      # Rows.
      rows = outputs[0].shape[1]
      image_height, image_width = input_image.shape[:2]
      # Resizing factor.
      x_factor = image_width / INPUT_WIDTH
      y_factor =  image_height / INPUT_HEIGHT
      # Iterate through detections
      for r in range(rows):
            row = outputs[0][0][r]
            confidence = row[4]
            # Discard bad detections and continue.
            if confidence >= CONFIDENCE_THRESHOLD:
                  classes_scores = row[5:]
                  # Get the index of max class score.
                  class_id = np.argmax(classes_scores)
                  #  Continue if the class score is above threshold.
                  if (classes_scores[class_id] > SCORE_THRESHOLD):
                        confidences.append(confidence)
                        class_ids.append(class_id)
                        cx, cy, w, h = row[0], row[1], row[2], row[3]
                        left = int((cx - w/2) * x_factor)
                        top = int((cy - h/2) * y_factor)
                        width = int(w * x_factor)
                        height = int(h * y_factor)
                        box = np.array([left, top, width, height])
                        boxes.append(box)

      # Perform non maximum suppression to eliminate redundant, overlapping boxes with lower confidences.
      indices = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
      for i in indices:
            box = boxes[i]
            left = box[0]
            top = box[1]
            width = box[2]
            height = box[3]             
            # Draw bounding box.             
            cv2.rectangle(input_image, (left, top), (left + width, top + height), BLACK, 3*THICKNESS)
            # Class label.                      
            label = "{}:{:.2f}".format(classes[class_ids[i]], confidences[i])             
            # Draw label.             
            draw_label(input_image, label, left, top)

            list_left.append(left)
            list_top.append(top)
            list_width.append(width)
            list_height.append(height)
            list_conf.append(confidences[i])
            list_class.append(classes[class_ids[i]])

      df_det_post = pd.DataFrame({'left': list_left, 'top': list_top, 'width': list_width, 'height': list_height, 'conf': list_conf, 'classes': list_class})
      print(df_det_post)

      # сохраним детекции в текстовый файл
      try:
            os.remove(f"*.txt")
      except:
            pass
      df_det_post.to_csv("*.txt", header=None, index=None, sep=' ', mode='a')
      print("Детекции сохранены: *.txt")

      return input_image


if __name__ == '__main__':

      classes = None
      with open(classesFile, 'rt') as f:
            classes = f.read().rstrip('\n').split('\n')
      
      net = cv2.dnn.readNet(modelWeights)

      detections = pre_process(frame, net)

      list_left = []
      list_top = []
      list_width = []
      list_height = []
      list_conf = []
      list_class = []
      img = post_process(frame.copy(), detections)
      
      # вычисление времени инференса
      t, _ = net.getPerfProfile()
      label = 'Inference time: %.2f ms' % (t * 1000.0 /  cv2.getTickFrequency())
      print(label)

      # сохраним изображение с результатами детектирования
      cv2.putText(img, label, (20, 40), FONT_FACE, FONT_SCALE,  (0, 0, 255), THICKNESS, cv2.LINE_AA)
      cv2.imwrite('*.png', img)
      print('Результат выполнения кода на изображении можно посмотреть здесь: *.png')


Почему-то этот код не сработал при тестировании на другом компьютере.

Зато работает следующий:

In [ ]:
import cv2
import numpy as np
import argparse
import pandas as pd
import onnxruntime as onnxrt
import cv2
import time

# Constants
INPUT_WIDTH = 640 # ширина входных изображений
INPUT_HEIGHT = 640 # высота входных изображений
SCORE_THRESHOLD = 0.45 # порог уверенности модели классификации
NMS_THRESHOLD = 0.45 # порог площади перекрытия для исключения лишних рамок
CONFIDENCE_THRESHOLD = 0.001 # порог уверенности модели детектора

# Text parameters.
FONT_FACE = cv2.FONT_HERSHEY_SIMPLEX
FONT_SCALE = 0.5
THICKNESS = 1

# Colors.
BLACK  = (0,0,0)
BLUE   = (255,178,50)
YELLOW = (0,255,255)


def draw_label(im, label, x, y):
    """Draw text onto image at location."""
    cv2.putText(im, label, (x, y), FONT_FACE, FONT_SCALE, BLACK, THICKNESS, cv2.LINE_AA)


def pre_process(input_image, onnx_session):
      start = time.time()
      input_image = cv2.dnn.blobFromImage(input_image, 1/255,  (INPUT_WIDTH, INPUT_HEIGHT), [0,0,0], 1, crop=False)
      onnx_inputs = {onnx_session.get_inputs()[0].name:input_image.astype(np.float32)}
      onnx_output = onnx_session.run(None, onnx_inputs)
      t = time.time() - start

      return onnx_output, t


def post_process(input_image, outputs):
    # Lists to hold respective values while unwrapping.
    class_ids = []
    confidences = []
    boxes = []
    # Rows.
    rows = outputs[0].shape[1]
    image_height, image_width = input_image.shape[:2]
    # Resizing factor.
    x_factor = image_width / INPUT_WIDTH
    y_factor =  image_height / INPUT_HEIGHT
    # Iterate through detections.
    for r in range(rows):
        row = outputs[0][0][r]
        confidence = row[4]
        # Discard bad detections and continue.
        if confidence >= CONFIDENCE_THRESHOLD:
            classes_scores = row[5:]
            # Get the index of max class score.
            class_id = np.argmax(classes_scores)
            #  Continue if the class score is above threshold.
            if (classes_scores[class_id] > SCORE_THRESHOLD):
                confidences.append(confidence)
                class_ids.append(class_id)
                cx, cy, w, h = row[0], row[1], row[2], row[3]
                left = int((cx - w/2) * x_factor)
                top = int((cy - h/2) * y_factor)
                width = int(w * x_factor)
                height = int(h * y_factor)
                box = np.array([left, top, width, height])
                boxes.append(box)

    # Perform non maximum suppression to eliminate redundant, overlapping boxes with lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, CONFIDENCE_THRESHOLD, NMS_THRESHOLD)
    for i in indices:
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]             
        # Draw bounding box.             
        cv2.rectangle(input_image, (left, top), (left + width, top + height), BLACK, 3*THICKNESS)
        # Class label.                      
        label = "{}:{:.2f}".format(classes[class_ids[i]], confidences[i])             
        # Draw label.             
        draw_label(input_image, label, left, top)

        list_left.append(left)
        list_top.append(top)
        list_width.append(width)
        list_height.append(height)
        list_conf.append(confidences[i])
        list_class.append(classes[class_ids[i]])

    df_det_post = pd.DataFrame({'left': list_left, 'top': list_top, 'width': list_width, 'height': list_height, 'conf': list_conf, 'classes': list_class})
    print(df_det_post)

    # сохраним детекции в текстовый файл
    try:
        os.remove(f"{folder_to_save}/{name.split('.')[0]}.txt")
    except:
        pass
    df_det_post.to_csv(f"{folder_to_save}/{name.split('.')[0]}.txt", header=None, index=None, sep=' ', mode='a')
    folder = folder_to_save.replace('/', '\\')
    print(f"Детекции сохранены здесь: {folder}\{name.split('.')[0]}.txt")

    return input_image


if __name__ == '__main__':

    # имена классов
    classesFile = 'names.names'
    classes = None
    with open(classesFile, 'rt') as f:
        classes = f.read().rstrip('\n').split('\n')

    # путь для сохранения результатов
    folder_to_save = '*'
    name = '*.png'
    
    # путь к изображению
    img_path = '*.png'
    frame = cv2.imread(img_path)

    onnx_session= onnxrt.InferenceSession("*.onnx")

    detections, t = pre_process(frame, onnx_session)
    
    list_left = []
    list_top = []
    list_width = []
    list_height = []
    list_conf = []
    list_class = []
    img = post_process(frame.copy(), detections)
      
    # вычисление времени инференса
    label = 'Inference time: %.2f ms' % (t * 1000)
    print(label)

    # сохраним изображение с результатами детектирования
    cv2.putText(img, label, (20, 40), FONT_FACE, FONT_SCALE,  (0, 0, 255), THICKNESS, cv2.LINE_AA)
    cv2.imwrite(f'{folder_to_save}/{name}', img)
    folder = folder_to_save.replace('/', '\\')
    print(f"Результат выполнения кода на изображении можно посмотреть здесь: {folder}\{name}")

## Из репозитория

In [ ]:
# !git clone https://github.com/ultralytics/yolov5
%cd yolov5
!python detect.py \
	--weights *.onnx \
    --source *.png \
    --conf-thres 0.25 \
    --iou-thres 0.45 \
	--img 640 \
    --project project_path \
    --save-txt \
    --save-conf